<a href="https://colab.research.google.com/github/Snowdenstyll/keno/blob/main/jupyterNotebooks/Keno_Lottery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [50]:
df = pd.read_csv("2024.csv")

In [51]:
df.head()
df.tail()
df.info()
df.describe()
df.drop(['PlayDate', 'AP'], axis=1, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 22 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   PlayDate  140 non-null    object
 1   AP        140 non-null    object
 2   N01       140 non-null    int64 
 3   N02       140 non-null    int64 
 4   N03       140 non-null    int64 
 5   N04       140 non-null    int64 
 6   N05       140 non-null    int64 
 7   N06       140 non-null    int64 
 8   N07       140 non-null    int64 
 9   N08       140 non-null    int64 
 10  N09       140 non-null    int64 
 11  N10       140 non-null    int64 
 12  N11       140 non-null    int64 
 13  N12       140 non-null    int64 
 14  N13       140 non-null    int64 
 15  N14       140 non-null    int64 
 16  N15       140 non-null    int64 
 17  N16       140 non-null    int64 
 18  N17       140 non-null    int64 
 19  N18       140 non-null    int64 
 20  N19       140 non-null    int64 
 21  N20       140 no

In [52]:
scaler = StandardScaler().fit(df.values)
transformed_dataset = scaler.transform(df.values)
transformed_df = pd.DataFrame(data=transformed_dataset, index=df.index)

In [53]:
# All our games
number_of_rows = df.values.shape[0]
# Amount of games we need to take into consideration for prediction
window_length = 10
number_of_features = df.values.shape[1]

X = np.empty([ number_of_rows - window_length, window_length, number_of_features], dtype=float)
y = np.empty([ number_of_rows - window_length, number_of_features], dtype=float)


In [54]:
for i in range(0, number_of_rows-window_length):
    X[i] = transformed_df.iloc[i : i+window_length, 0 : number_of_features]
    y[i] = transformed_df.iloc[i+window_length : i+window_length+1, 0 : number_of_features]

In [55]:
# Recurrent Neural Netowrk (RNN) with Long Short Term Memory (LSTM)
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Dropout
#batch_size = 40
batch_size = 70

# Initialising the RNN
model = Sequential()
# Adding the input layer and the LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a first Dropout layer
model.add(Dropout(0.2))
# Adding a second LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a second Dropout layer
model.add(Dropout(0.2))
# Adding a third LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a fourth LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = False)))
# Adding a fourth Dropout layer
model.add(Dropout(0.2))
# Adding the first output layer
model.add(Dense(70))
# Adding the last output layer
model.add(Dense(number_of_features))

In [56]:
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

model.compile(optimizer=Adam(learning_rate=0.0001), loss ='mse', metrics=['accuracy'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.95, random_state=95)

# Train the model
history = model.fit(X, y, batch_size=batch_size, epochs=1000, verbose=2)
#history = model.fit(X_train, y_train, batch_size=100, epochs=600, verbose=2)

Epoch 1/1000
2/2 - 17s - loss: 1.0174 - accuracy: 0.0769 - 17s/epoch - 9s/step
Epoch 2/1000
2/2 - 0s - loss: 1.0135 - accuracy: 0.0615 - 54ms/epoch - 27ms/step
Epoch 3/1000
2/2 - 0s - loss: 1.0089 - accuracy: 0.0769 - 53ms/epoch - 27ms/step
Epoch 4/1000
2/2 - 0s - loss: 1.0067 - accuracy: 0.0846 - 47ms/epoch - 24ms/step
Epoch 5/1000
2/2 - 0s - loss: 1.0028 - accuracy: 0.1000 - 41ms/epoch - 20ms/step
Epoch 6/1000
2/2 - 0s - loss: 0.9979 - accuracy: 0.1000 - 40ms/epoch - 20ms/step
Epoch 7/1000
2/2 - 0s - loss: 0.9952 - accuracy: 0.1000 - 41ms/epoch - 21ms/step
Epoch 8/1000
2/2 - 0s - loss: 0.9920 - accuracy: 0.1077 - 40ms/epoch - 20ms/step
Epoch 9/1000
2/2 - 0s - loss: 0.9868 - accuracy: 0.0923 - 33ms/epoch - 16ms/step
Epoch 10/1000
2/2 - 0s - loss: 0.9823 - accuracy: 0.1000 - 32ms/epoch - 16ms/step
Epoch 11/1000
2/2 - 0s - loss: 0.9759 - accuracy: 0.0846 - 31ms/epoch - 15ms/step
Epoch 12/1000
2/2 - 0s - loss: 0.9717 - accuracy: 0.0923 - 31ms/epoch - 16ms/step
Epoch 13/1000
2/2 - 0s - lo

In [57]:
loss, accuracy = model.evaluate(X, y, verbose=1)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

5/5 [==============================] - 2s 9ms/step - loss: 0.0029 - accuracy: 0.9462
Test Loss: 0.002908332971855998, Test Accuracy: 0.9461538195610046
4/4 [==============================] - 0s 10ms/step - loss: 0.0029 - accuracy: 0.9597
Test Loss: 0.0029229235369712114, Test Accuracy: 0.9596773982048035


In [58]:
#history = model.fit(X_train, y_train, batch_size=50, epochs=100, verbose=2)
#history = model.fit(X_train, y_train, batch_size=50, epochs=100, verbose=2)

In [59]:
df1 = df.copy()
to_predict = df1.tail(window_length)
to_predict.drop([to_predict.index[-1]],axis=0, inplace=True)
to_predict

<ipython-input-59-efb52fef0daf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_predict.drop([to_predict.index[-1]],axis=0, inplace=True)


,N01,N02,N03,N04,N05,N06,N07,N08,N09,N10,N11,N12,N13,N14,N15,N16,N17,N18,N19,N20
130,4,11,12,20,22,23,31,36,41,44,45,49,52,54,58,59,60,61,64,68
131,4,12,24,26,29,31,33,39,40,42,50,51,52,53,54,58,59,63,68,70
132,1,4,8,9,15,16,21,26,28,30,38,39,43,47,48,55,61,63,64,66
133,2,12,13,14,17,21,25,28,29,30,34,40,47,56,58,64,65,66,67,68
134,4,9,10,11,13,16,18,21,22,34,35,36,39,44,49,50,53,58,60,66
135,2,3,7,8,10,11,13,22,24,25,31,37,38,39,46,49,55,58,59,62
136,2,4,6,9,11,13,19,23,26,30,35,36,42,44,50,55,58,59,64,66
137,8,10,12,13,20,21,26,31,34,35,37,45,46,52,54,58,62,65,68,69
138,1,4,7,9,10,21,33,36,37,38,40,42,47,51,53,57,61,64,65,70


In [60]:
prediction = np.array(df.tail(1))

x_next = scaler.transform(to_predict)
y_pred = model.predict(np.array([x_next]))

temp = np.array(np.unique(scaler.inverse_transform(y_pred).astype(int)[0]))
combined_array = np.concatenate((temp, np.array(prediction[0])))
winning_numbers = np.bincount(combined_array)
winners = np.unique(combined_array)

uniques, uniq_idx, counts = np.unique(combined_array,return_index=True,return_counts=True)
duplicates = combined_array[uniq_idx[counts>=2] ]  # <--- Get duplicates

top_5_indices = np.argsort(y_pred[0], axis=0)[::-1][:-10]
top_5_numbers = []

winning_indices = np.where(np.isin(prediction[0], duplicates))[0]

# Print the winning indices
print("The winning indices:", winning_indices)

for index in top_5_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    top_5_numbers.append(number)

print("The top 5 indices:", top_5_indices)
print("The top 5 numbers:", top_5_numbers)

print("The predicted numbers for the lottery game are:", temp)
print("The actual numbers in the last lottery game were:", prediction[0])
print("\n")
print("winners:", duplicates)
print(len(duplicates)/20)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


1/1 [==============================] - 5s 5s/step
The winning indices: [ 0  4  7  9 12 13 14 15 17 18]
The top 5 indices: [ 6  5  3  4  7 10  8 12 11 13]
The top 5 numbers: [28, 25, 20, 23, 29, 39, 30, 45, 42, 47]
The predicted numbers for the lottery game are: [ 1  6  9 17 19 23 27 29 31 38 41 45 47 48 53 56 58 62 64]
The actual numbers in the last lottery game were: [ 1  8 11 20 23 25 28 29 30 31 39 42 45 47 48 53 55 58 62 63]


winners: [ 1 23 29 31 45 47 48 53 58 62]
0.5


# Predict the next set of numbers

In [61]:
next = df.copy()
next = next.tail(window_length)
next = np.array(next)

x_next = scaler.transform(next)
y_next_pred = model.predict(np.array([x_next]))
prediction = scaler.inverse_transform(y_next_pred).astype(int)
print("The predicted numbers for the lottery game:", temp)
print("\n")
top_5_indices = np.argsort(y_next_pred[0], axis=0)[::-1][:5]
top_5_numbers = []
for index in top_5_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    top_5_numbers.append(number)

print("The top 5 indices", top_5_indices)
print("The top 5 numbers:", top_5_numbers)
print("\n")

winning_numbers = []
for index in winning_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    winning_numbers.append(number)

top_10_indices = np.argsort(y_next_pred[0], axis=0)[::-1][:10]
top_10_numbers = []
for index in top_10_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]

    # Append the number to the top_10_numbers array
    top_10_numbers.append(number)
print("The top 10 indices", top_10_indices)
print("The top 10 numbers:", top_10_numbers)
print("\n")

print("winning indices", winning_indices)
print("winning numbers", winning_numbers)


1/1 [==============================] - 3s 3s/step
The predicted numbers for the lottery game: [ 1  6  9 17 19 23 27 29 31 38 41 45 47 48 53 56 58 62 64]


The top 5 indices [ 6  7  8 16 15]
The top 5 numbers: [27, 30, 32, 58, 55]


The top 10 indices [ 6  7  8 16 15 10  5 17 18 12]
The top 10 numbers: [27, 30, 32, 58, 55, 38, 21, 60, 64, 44]


winning indices [ 0  4  7  9 12 13 14 15 17 18]
winning numbers [0, 16, 30, 33, 44, 46, 49, 55, 60, 64]


In [62]:
next = df.copy()
next = next.tail(1)
next = np.array(next)

x_next = scaler.transform(next)
y_next_pred = model.predict(np.array([x_next]))

# Get the actual values of the top 10 predictions
prediction = scaler.inverse_transform(y_next_pred).astype(int)

# Get the indices of the top 10 predictions (change axis=0 if predictions are along a different axis)
top_10_indices = np.argsort(y_next_pred[0], axis=0)[::-1][:10]
top_5_indices = np.argsort(y_next_pred[0], axis=0)[::-1][:5]

top_10_numbers = []
top_5_numbers = []

# index [0,2,4] usually are correct
# Loop through each index in top_10_indices
for index in top_10_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]

    # Append the number to the top_10_numbers array
    top_10_numbers.append(number)

for index in top_5_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]

    # Append the number to the top_10_numbers array
    top_5_numbers.append(number)

winning_numbers = []
for index in winning_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    winning_numbers.append(number)

print("winning indices", winning_indices)
print("winning numbers", winning_numbers)
print("\n")

print("The top 5 indices", top_5_indices)
print("The top 5 numbers:", top_5_numbers)
print("\n")

print("The top 10 indices", top_10_indices)
print("The top 10 numbers:", top_10_numbers)
print("\n")

print("The predicted numbers:", prediction)


1/1 [==============================] - 0s 20ms/step
winning indices [ 0  4  7  9 12 13 14 15 17 18]
winning numbers [3, 16, 26, 33, 43, 46, 49, 53, 59, 63]


The top 5 indices [8 7 0 9 6]
The top 5 numbers: [30, 26, 3, 33, 23]


The top 10 indices [ 8  7  0  9  6 17 16  3 15  1]
The top 10 numbers: [30, 26, 3, 33, 23, 59, 56, 13, 53, 6]


The predicted numbers: [[ 3  6  9 13 16 19 23 26 30 33 36 39 43 46 49 53 56 59 63 67]]
